# Import dependencies

In [14]:
import os
import sys
import pandas as pd
import re
import pickle
import ast

# Obtener la ruta del directorio padre
parent_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Añadir la ruta del directorio padre al path de Python
sys.path.append(parent_path)
from connect import *
from dataStructure import *

# Read data

In [2]:
parent_path

'c:\\trabajo\\tes-obleas'

## Read source excel

In [8]:
PATH_EXCEL = os.path.join(parent_path,'rawData','TES3Z.xlsx')
df_excel = pd.read_excel(PATH_EXCEL,index_col=0)

In [43]:
def extrae_parametros(cadena:str)->List:
    
    patron1 = r'\[\[(.*?)\]\]'
    patron2 = r'\[(.*?)\]'
    patron3 = r'\((.*?)\)'

    elementos = re.findall(patron1, cadena)
    elementos = '['+elementos[0]+']'
    elementos = re.findall(patron2, elementos)
    elementos = [re.findall(patron3, i) for i in elementos]
    elementos = [[list(map(str.strip,elemento.split(','))) for elemento in grupo] for grupo in elementos]
    return elementos



In [2]:
data_path = os.path.join(parent_path,'OutputData')
files = os.listdir(data_path)
file = [f for f in files if 'df_agg' in f][0]
df_file = os.path.join(data_path,file)

with open(df_file, 'rb') as handle:
    df = pickle.load(handle)

# Preparing Data

## New document

In [3]:
new_tes = []
for n, row in df.iterrows():
    var = Tes(
        tesName = row['tesName'],
        tags = list(row['tags']),
        fabrication = Fabrication(
            wafer = Wafer(
                waferName = row['tesName'].split('_')[0],
                material = 'Si',
                orientation = '100',
                divisions = ['Q1','Q2','Q3','Q4'],
                chipsOnWafer= [f'Chip_{i}' for i in range(23,77)],
                image = os.path.join(parent_path,'Images','wafer_3Z.jpg')
                ),
            absorber = None if row['absorber'] == 'no' else [Layer(
                    material = 'Au',
                    process = 'electroplating',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 2, units = 'um'),
                    width = PairValueUnit(quantity = 240, units = 'um'),
                    length = PairValueUnit(quantity = 240, units = 'um')
                ),Layer(
                    material = 'Bi',
                    process = 'electroplating',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 3.5, units = 'um'),
                    width = PairValueUnit(quantity = 240, units = 'um'),
                    length = PairValueUnit(quantity = 240, units = 'um')
                )],
            membrane = None if row['membrane'] == 'no' else Layer(
                    material = 'Si3N4',
                    process = 'Sputtering',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 0.5, units = 'um'),
                    width = PairValueUnit(quantity = 250, units = 'um'),
                    length = PairValueUnit(quantity = 250, units = 'um')
                ),
            pads = [Pads(
                material = 'Mo',
                orientation = 'amorphous',
                process = 'sputtering-dc',
                thickness = PairValueUnit(quantity = 15, units = 'um') 
            ),Pads(
                material = 'Nb',
                orientation = 'amorphous',
                process = 'sputtering-dc',
                thickness = PairValueUnit(quantity = 0.15, units = 'um') 
            )],
            stack = [Layer(
                    material = 'Mo',
                    process = 'sputtering-dc',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackMoT'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                ),Layer(
                    material = 'Au',
                    process = 'sputtering-dc',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackAu1T'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                ),Layer(
                    material = 'Au',
                    process = 'e-beam_evaporation',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackAu2T'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                )]
        )
    ).dict()
    new_tes.append(var)

In [4]:
df.head()

,tesName,tags,stackw,stackl,stackMoT,stackAu1T,stackAu2T,absorber,membrane,RN,TC0
0,3Z10_33_34,"(EB, Funcional)",30,16,40,15,200,no,no,16.0,[92]
1,3Z10_53_13,"(EB, Funcional)",16,10,40,15,200,no,no,19.0,[100]
2,3Z10_53_33,"(EB, Funcional)",10,16,40,15,200,no,no,48.0,[85]
3,3Z10_64_14,"(EB, Funcional)",16,8,40,15,200,no,no,15.0,[170]
4,3Z10_64_41,"(EB, Funcional)",16,24,40,15,200,no,no,43.0,[90]


# Pushing data

In [6]:
objeto = DatabaseTes(True)
objeto.connect()
objeto.collection('tes')
db = objeto.collec

## Insert new documents

In [7]:
db.insert_many(new_tes)